In [1]:
import pandas as pd
import json

In [ ]:
import tensorflow as tf
tf.sysconfig.get_build_info() 

: 

In [2]:
split2data = {
    "val": "infoseek_data/infoseek_val.jsonl",
    "val_withkb": "infoseek_data/infoseek_val_withkb.jsonl",
    "test": "infoseek_data/infoseek_test.jsonl",
    "human": "infoseek_data/infoseek_human.jsonl",
    "train": "infoseek_data/infoseek_train.jsonl",
    "train_withkb": "infoseek_data/infoseek_train_withkb.jsonl"
}

qtype = {
    "val": "infoseek_qtype/infoseek_val.jsonl",
    "val_mini": "infoseek_qtype/infoseek_val_mini.jsonl",
    "test": "infoseek_qtype/infoseek_test.jsonl",
    "test_qtype": "infoseek_qtype/infoseek_test_qtype.jsonl",
    "human": "infoseek_qtype/infoseek_human.jsonl",
    "train": "infoseek_qtype/infoseek_train.jsonl"
}

In [90]:
train_df = merge(qtype['train'], split2data["train_withkb"])
val_df = merge(qtype['val'], split2data["val_withkb"])
test_df = js2df(qtype['test'])

In [91]:
train_df_new = pd.read_csv("infoseek/train.csv")
val_df_new = pd.read_csv("infoseek/val.csv")
test_df_new = pd.read_csv("infoseek/test.csv")

In [92]:
train_df = train_df[train_df["data_id"].isin(train_df_new["data_id"])]
val_df = val_df[val_df["data_id"].isin(val_df_new["data_id"])]
test_df = test_df[test_df["data_id"].isin(test_df_new["data_id"])]

In [78]:
train_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_train_00000000,oven_01963180,Which place is this animal endemic to?,[People's Republic of China],"[cn, People's Republic of China, China, Mainla...",train,String,Q33602,Giant panda
1,infoseek_train_00000001,oven_03952028,What is the mohs' hardness of this material?,[7.6],"[{'wikidata': 7.6, 'range': [6.84, 8.36]}]",train,Numerical,Q43513,Emerald
2,infoseek_train_00000002,oven_01857671,What is the conservation status of this animal...,[Endangered],"[Endangered species, Endangered, EN]",train,String,Q192710,Malayan tapir
3,infoseek_train_00000003,oven_02959375,Who is the manufacturer of this vehicle?,[AM General],"[Am General, AM General LLC, AM General, AM Ge...",train,String,Q217046,Humvee
4,infoseek_train_00000004,oven_04416546,What fields are the person in the image specia...,[baking],[baking],train,String,Q160131,Baker
...,...,...,...,...,...,...,...,...,...
934043,infoseek_train_00934043,oven_00670496,How many days does the gestation of this anima...,[30.0-32.0],"[{'wikidata': 31.0, 'range': [30.0, 32.0]}]",train,Numerical,Q221612,Groundhog
934044,infoseek_train_00934044,oven_01683806,What is the oldest age of this animal?,[25],"[{'wikidata': 25.0, 'range': [22.5, 27.5000000...",train,Numerical,Q188657,Striped hyena
934045,infoseek_train_00934045,oven_00270092,What is the minimum number of players of a gam...,[2],"[{'wikidata': 2.0, 'range': [1.8, 2.2]}]",train,Numerical,Q7291,Badminton
934046,infoseek_train_00934046,oven_04651005,What is the source that produces this food?,[Quercus],"[oaks, oak, oak tree, the oak genus, Quercus]",train,String,Q3914781,Acorn


In [93]:
len(train_df), len(val_df), len(test_df)

(933972, 73545, 347679)

In [94]:
# idx
train_remaining_idx = pd.read_csv("infoseek/train_remaining.csv", index_col="data_id").index.tolist()
val_idx = pd.read_csv("infoseek/val.csv", index_col="data_id").index.tolist()
val_seen_idx = pd.read_csv("infoseek/val_seen.csv", index_col="data_id").index.tolist()
test_idx = pd.read_csv("infoseek/test.csv", index_col="data_id").index.tolist()
test_seen_idx = pd.read_csv("infoseek/test_seen.csv", index_col="data_id").index.tolist()

train_remaining_10_idx = pd.read_csv("infoseek/train_remaining_10%.csv", index_col="data_id").index.tolist()
val_10_idx = pd.read_csv("infoseek/val_unseen_10%.csv", index_col="data_id").index.tolist()
val_seen_10_idx = pd.read_csv("infoseek/val_seen_10%.csv", index_col="data_id").index.tolist()

# image_id
train_remaining_image_id = pd.read_csv("infoseek/train_remaining.csv").sort_values('data_id')['image_id'].tolist()
val_image_id = pd.read_csv("infoseek/val.csv").sort_values('data_id')['image_id'].tolist()
val_seen_image_id = pd.read_csv("infoseek/val_seen.csv").sort_values('data_id')['image_id'].tolist()
test_image_id = pd.read_csv("infoseek/test.csv").sort_values('data_id')['image_id'].tolist()
test_seen_image_id = pd.read_csv("infoseek/test_seen.csv").sort_values('data_id')['image_id'].tolist()

train_remaining_10_image_id = pd.read_csv("infoseek/train_remaining_10%.csv").sort_values('data_id')['image_id'].tolist()
val_10_image_id = pd.read_csv("infoseek/val_unseen_10%.csv").sort_values('data_id')['image_id'].tolist()
val_seen_10_image_id = pd.read_csv("infoseek/val_seen_10%.csv").sort_values('data_id')['image_id'].tolist()

# filter train_df with train_remaining_idx
train_remaining = train_df[train_df['data_id'].isin(train_remaining_idx)].sort_values('data_id')
val_unseen = val_df[val_df['data_id'].isin(val_idx)].sort_values('data_id')
val_seen = train_df[train_df['data_id'].isin(val_seen_idx)].sort_values('data_id')
test_unseen = test_df[test_df['data_id'].isin(test_idx)].sort_values('data_id')
test_seen = train_df[train_df['data_id'].isin(test_seen_idx)].sort_values('data_id')

train_remaining_10 = train_df[train_df['data_id'].isin(train_remaining_10_idx)].sort_values('data_id')
val_unseen_10 = val_df[val_df['data_id'].isin(val_10_idx)].sort_values('data_id')
val_seen_10 = train_df[train_df['data_id'].isin(val_seen_10_idx)].sort_values('data_id')

val_seen['data_split'] = 'val_seen'
test_seen['data_split'] = 'test_seen'
val_seen_10['data_split'] = 'val_seen'

# check
# list ascending order
assert train_remaining_image_id == train_remaining['image_id'].tolist()
assert val_image_id == val_unseen['image_id'].tolist()
assert val_seen_image_id == (val_seen['image_id'].tolist())
assert test_image_id == (test_unseen['image_id'].tolist())
assert test_seen_image_id == (test_seen['image_id'].tolist())

assert train_remaining_10_image_id == (train_remaining_10['image_id'].tolist())
assert val_10_image_id == (val_unseen_10['image_id'].tolist())
assert val_seen_10_image_id == (val_seen_10['image_id'].tolist())


In [95]:
len(val_image_id), len(val_unseen['image_id'].tolist())

(73545, 73545)

In [96]:
train_remaining.to_json("infoseek/infoseek_train.jsonl", orient="records", lines=True)
val_unseen.to_json("infoseek/infoseek_val_unseen.jsonl", orient="records", lines=True)
val_seen.to_json("infoseek/infoseek_val_seen.jsonl", orient="records", lines=True)
test_unseen.to_json("infoseek/infoseek_test_unseen.jsonl", orient="records", lines=True)
test_seen.to_json("infoseek/infoseek_test_seen.jsonl", orient="records", lines=True)

train_remaining_10.to_json("infoseek/infoseek_train_10%.jsonl", orient="records", lines=True)
val_unseen_10.to_json("infoseek/infoseek_val_unseen_10%.jsonl", orient="records", lines=True)
val_seen_10.to_json("infoseek/infoseek_val_seen_10%.jsonl", orient="records", lines=True)

In [4]:
def load_jsonl(path):
    """Load a JSONL file into a list of Dict[strionaries."""
    data = []
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [3]:
def prepare_qid2example(
    reference,
    reference_qtype,
    ):
    """Convert reference to qid2example dictionary."""
    qid2qtype = dict()
    for qtype in reference_qtype:
        qid = qtype["data_id"]
        qid2qtype[qid] = qtype["question_type"]

    qid2example = dict()
    for r in reference:
        qid = r['data_id']
        q_type = qid2qtype[qid].lower()
        if q_type == 'numerical':
        # Process numerical answer:
        # "answer_eval": [{"wikidata": 1.0, "range": [0.9, 1.1]}]
        # --> "answer_eval": [0.9, 1.1]
            if isinstance(r['answer_eval'], list):
                ans_eval = r['answer_eval'][0]['range']
            else:
                ans_eval = r['answer_eval']['range']
            r['answer_eval'] = [str(ans) for ans in ans_eval][:2]

        qid2example[qid] = r
        qid2example[qid]["question_type"] = qid2qtype[qid]
    return qid2example

In [5]:
prepare_qid2example(load_jsonl("infoseek_data/infoseek_val.jsonl"), load_jsonl("infoseek_data/infoseek_val_qtype.jsonl"))

{'infoseek_val_00000000': {'data_id': 'infoseek_val_00000000',
  'image_id': 'oven_04990048',
  'question': 'What is the objective of this object?',
  'answer': ['energy transformation'],
  'answer_eval': ['energy conversion', 'energy transformation', 'conversion'],
  'data_split': 'val_unseen_question',
  'question_type': 'String'},
 'infoseek_val_00000001': {'data_id': 'infoseek_val_00000001',
  'image_id': 'oven_04962816',
  'question': 'What is the conservation status of this animal? (The status is assigned by the international union for conservation of nature. Choose one among Endangered,Least Concern,Critically Endangered,extinct species,extinct in the wild,Vulnerable,Near Threatened,Data Deficient)',
  'answer': ['Endangered'],
  'answer_eval': ['Endangered species', 'Endangered', 'EN'],
  'data_split': 'val_unseen_question',
  'question_type': 'String'},
 'infoseek_val_00000002': {'data_id': 'infoseek_val_00000002',
  'image_id': 'oven_04967661',
  'question': 'How many days do

In [7]:
tmp1 = prepare_qid2example(load_jsonl("infoseek_qtype/infoseek_val.jsonl"), load_jsonl("infoseek_data/infoseek_val_qtype.jsonl"))
tmp2 = prepare_qid2example(load_jsonl("infoseek_qtype/infoseek_val.jsonl"), load_jsonl("infoseek_qtype/infoseek_val.jsonl"))
tmp1 == tmp2

True

In [100]:
load_jsonl("infoseek/infoseek_val_seen.jsonl")

[{'data_id': 'infoseek_train_00000019',
  'image_id': 'oven_03183340',
  'question': 'What is the country of origin of this material?',
  'answer': ['India'],
  'answer_eval': ['in',
   'Bharat',
   'India',
   '🇮🇳',
   'Republic of India',
   'Hindustan',
   'IN',
   'IND',
   'Bharatvarsh',
   'Aryavratt'],
  'data_split': 'val_seen',
  'question_type': 'String',
  'entity_id': 'Q725457',
  'entity_text': 'Muslin'},
 {'data_id': 'infoseek_train_00000027',
  'image_id': 'oven_03088768',
  'question': "What is this person's place of birth?",
  'answer': ['Brooklyn'],
  'answer_eval': ['Brooklyn, New York City', 'Brooklyn, New York', 'Brooklyn'],
  'data_split': 'val_seen',
  'question_type': 'String',
  'entity_id': 'Q202878',
  'entity_text': 'Mae West'},
 {'data_id': 'infoseek_train_00000045',
  'image_id': 'oven_04014713',
  'question': 'What is the immediately prior item that this object follows in a series?',
  'answer': ['breakfast'],
  'answer_eval': ['breakfast'],
  'data_split

In [97]:
len(train_remaining), len(val_unseen), len(val_seen), len(test_unseen), len(test_seen), len(train_remaining_10), len(val_unseen_10), len(val_seen_10)

(790806, 73545, 71583, 347679, 71583, 76974, 7354, 7158)

In [84]:
len(train_remaining), len(val_unseen), len(val_seen), len(test_unseen), len(test_seen), len(train_remaining_10), len(val_unseen_10), len(val_seen_10)

(790806, 73545, 71583, 347679, 71583, 76974, 5489, 7158)

In [85]:
val_unseen['data_split'].value_counts()

data_split
val_unseen_entity      54889
val_unseen_question    18656
Name: count, dtype: int64

In [86]:
len(val_unseen)

73545

In [81]:
# difference between test_seen_image_id and test_seen['image_id'].tolist()
val_unseen_10.sort_values('data_id').reset_index(drop=True)

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_val_00018666,oven_05009923,What is the total quantity of produced items f...,[217],"[{'wikidata': 217.0, 'range': [195.3, 238.7000...",val_unseen_entity,Numerical,Q937949,Dornier 328
1,infoseek_val_00018684,oven_05026052,Which diocese does this building belong to?,[Diocese of Stockholm],"[Diocese of Stockholm, Stockholms stift]",val_unseen_entity,String,Q1477827,"Saint James's Church, Stockholm"
2,infoseek_val_00018700,oven_05064165,What is the country of origin of this food?,[France],"[FRA, FR, fr, French Republic, the Hexagon, La...",val_unseen_entity,String,Q527323,Coq au vin
3,infoseek_val_00018710,oven_05018712,What country does this building belong to?,[Singapore],"[Lion City, 🇸🇬, SGP, Republic of Singapore, Ga...",val_unseen_entity,String,Q2428845,"Sri Mariamman Temple, Singapore"
4,infoseek_val_00018713,oven_05013739,How far in nautical mile can people see the li...,[20],"[{'wikidata': 20.0, 'range': [18.0, 22.0]}]",val_unseen_entity,Numerical,Q11321434,Niechorze Lighthouse
...,...,...,...,...,...,...,...,...,...
5484,infoseek_val_00073562,oven_05018507,Which significant event was held in this build...,[opening ceremony],"[opening ceremony, opening ceremonies]",val_unseen_entity,String,Q160700,Memorial to the Murdered Jews of Europe
5485,infoseek_val_00073576,oven_05020645,What is this building named after?,[Franz von Lenbach],"[Professor Franz von Lenbach, lenbach fr. v., ...",val_unseen_entity,String,Q262234,Lenbachhaus
5486,infoseek_val_00073580,oven_04948333,Who is the director of this building?,[David Hare],"[David Hare, Sir David Hare]",val_unseen_entity,String,Q3487748,Strapless
5487,infoseek_val_00073586,oven_05024972,What is the sea level in metre of this building?,[11.7],"[{'wikidata': 11.7, 'range': [10.53, 12.870000...",val_unseen_entity,Numerical,Q752721,Carew Castle


In [61]:
set(train_remaining_10_image_id) == set(train_remaining_10['image_id'].tolist())

True

In [70]:
train_remaining_image_id.sort()
train_remaining_image_id

['oven_00000000',
 'oven_00000000',
 'oven_00000001',
 'oven_00000001',
 'oven_00000002',
 'oven_00000003',
 'oven_00000004',
 'oven_00000004',
 'oven_00000005',
 'oven_00000005',
 'oven_00000006',
 'oven_00000007',
 'oven_00000008',
 'oven_00000009',
 'oven_00000009',
 'oven_00000010',
 'oven_00000011',
 'oven_00000012',
 'oven_00000013',
 'oven_00000013',
 'oven_00000014',
 'oven_00000014',
 'oven_00000015',
 'oven_00000016',
 'oven_00000017',
 'oven_00000017',
 'oven_00000018',
 'oven_00000018',
 'oven_00000019',
 'oven_00000019',
 'oven_00000020',
 'oven_00000021',
 'oven_00000022',
 'oven_00000023',
 'oven_00000023',
 'oven_00000024',
 'oven_00000024',
 'oven_00000026',
 'oven_00000027',
 'oven_00000027',
 'oven_00000028',
 'oven_00000028',
 'oven_00000029',
 'oven_00000030',
 'oven_00000031',
 'oven_00000032',
 'oven_00000033',
 'oven_00000034',
 'oven_00000035',
 'oven_00000035',
 'oven_00000036',
 'oven_00000037',
 'oven_00000037',
 'oven_00000038',
 'oven_00000039',
 'oven_000

In [50]:
train_remaining['image_id'].tolist()

['oven_01963180',
 'oven_03952028',
 'oven_01857671',
 'oven_03002067',
 'oven_03957581',
 'oven_00077923',
 'oven_04148759',
 'oven_00057589',
 'oven_02976344',
 'oven_00155366',
 'oven_02762378',
 'oven_03721368',
 'oven_00106760',
 'oven_03871483',
 'oven_04633031',
 'oven_00022576',
 'oven_01614994',
 'oven_01519940',
 'oven_00003528',
 'oven_03088296',
 'oven_04335061',
 'oven_02563072',
 'oven_01810198',
 'oven_02282874',
 'oven_00649331',
 'oven_01223607',
 'oven_03890288',
 'oven_03952005',
 'oven_02976478',
 'oven_00545944',
 'oven_01871954',
 'oven_04168394',
 'oven_03537845',
 'oven_02788852',
 'oven_00079743',
 'oven_01660031',
 'oven_03639456',
 'oven_00052584',
 'oven_01669186',
 'oven_03952006',
 'oven_03640001',
 'oven_00108395',
 'oven_04081591',
 'oven_00180914',
 'oven_02901271',
 'oven_03024478',
 'oven_02728594',
 'oven_01265051',
 'oven_04413015',
 'oven_01771071',
 'oven_00015180',
 'oven_03951880',
 'oven_00827413',
 'oven_04325893',
 'oven_04148474',
 'oven_019

In [48]:
val_seen['image_id'].tolist()

['oven_03183340',
 'oven_03088768',
 'oven_04014713',
 'oven_01802338',
 'oven_04196986',
 'oven_02240376',
 'oven_00928760',
 'oven_03652928',
 'oven_03686206',
 'oven_03177161',
 'oven_04121634',
 'oven_03988127',
 'oven_02064953',
 'oven_02326512',
 'oven_00499724',
 'oven_00729218',
 'oven_00148124',
 'oven_00815588',
 'oven_03182500',
 'oven_03612377',
 'oven_03158347',
 'oven_00675947',
 'oven_01154528',
 'oven_03649760',
 'oven_04444878',
 'oven_01647366',
 'oven_03169971',
 'oven_02896181',
 'oven_03500538',
 'oven_03564764',
 'oven_02894008',
 'oven_02096626',
 'oven_03507631',
 'oven_00180815',
 'oven_03795589',
 'oven_03146166',
 'oven_02471625',
 'oven_01067170',
 'oven_03220681',
 'oven_00504791',
 'oven_00904058',
 'oven_04242809',
 'oven_03375130',
 'oven_01725744',
 'oven_03158025',
 'oven_04354946',
 'oven_00202165',
 'oven_03249199',
 'oven_01464688',
 'oven_03088444',
 'oven_02675169',
 'oven_03001621',
 'oven_03008743',
 'oven_04313709',
 'oven_01666307',
 'oven_011

In [20]:
train_remaining_data_id[100]

'infoseek_train_00000121'

In [21]:
train_remaining['data_id'].tolist()[100]

'infoseek_train_00000100'

In [6]:
train_image_id = []
val_image_id = []
val_data_split = []

with open(split2data["train"], "r", encoding="utf-8") as f:    
    for line in f:
        line = json.loads(line)
        train_image_id.append(line["image_id"])
len(train_image_id)

934048

In [4]:
with open(split2data["val"], 'r') as f:
    for line in f:
        line = json.loads(line)
        val_image_id.append(line["image_id"])
        val_data_split.append(line["data_split"])

In [5]:
len(val_image_id)

73620

In [10]:
train_entity_id = []
with open(split2data["train_withkb"], "r", encoding="utf-8") as f:    
    for line in f:
        line = json.loads(line)
        train_entity_id.append(line["entity_id"])

In [11]:
train_entity_id = list(set(train_entity_id))
len(train_entity_id)

5549

In [12]:
val_entity_id = []
with open(split2data["val_withkb"], "r", encoding="utf-8") as f:    
    for line in f:
        line = json.loads(line)
        val_entity_id.append(line["entity_id"])
val_entity_id = list(set(val_entity_id))
len(val_entity_id)

1794

# Statistics Check

In [3]:
# convert jsonl to dataframe
def js2df(file_path):
    # Initialize an empty list to store the JSON objects
    data = []

    # Open the file and read each line
    with open(file_path, 'r') as file:
        for line in file:
            # Convert each line to a dictionary
            json_object = json.loads(line)
            data.append(json_object)

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    return df

# convert dataframe to jsonl
def df2js(df, file_path):
    with open(file_path, 'w') as file:
        for index, row in df.iterrows():
            json.dump(row.to_dict(), file)
            file.write('\n')

In [4]:
def merge(df1, df2, key="data_id"):
    df1 = js2df(df1)
    df2 = js2df(df2)
    merged_df = pd.merge(df1, df2, on=key, how='inner')
    return merged_df

In [5]:
train_df = merge(qtype['train'], split2data["train_withkb"])
val_df = merge(qtype['val'], split2data["val_withkb"])
test_df = js2df(qtype['test'])

In [6]:
test_df = js2df(qtype['test'])
test_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type
0,infoseek_test_00000000,oven_05494604,What place inflows lake?,"[Aare, Lütschine]","[Lütschine, Lutschine, Aare, Aar]",test_unseen_question,String
1,infoseek_test_00000001,oven_05651031,What is the country of origin of this animal?,"[Scotland, United Kingdom]","[U K, Great Britain, Britain, The United Kingd...",test_unseen_question,String
2,infoseek_test_00000002,oven_05717674,What is the brand of this vehicle?,[Ford],"[Ford brand of vehicles, Ford]",test_unseen_question,String
3,infoseek_test_00000003,oven_05528107,Who is the owner of this building?,[Cleveland],"[Cleveland, Ohio, Cleveland, Cleveland, OH]",test_unseen_question,String
4,infoseek_test_00000004,oven_05504893,How many spans does this bridge have?,[1],"[{'wikidata': 1.0, 'range': [0.9, 1.1]}]",test_unseen_question,Numerical
...,...,...,...,...,...,...,...
347975,infoseek_test_00347975,oven_05161211,Who commissioned this building?,[Damian Hugo Philipp von Schönborn],[Damian Hugo Philipp von Schönborn],test_unseen_entity,String
347976,infoseek_test_00347976,oven_05279591,What is the closest parent taxonomy of this an...,[Callospermophilus],[Callospermophilus],test_unseen_entity,String
347977,infoseek_test_00347977,oven_05454536,What country does this food belong to?,[Italy],"[Italy, Italian Republic, 🇮🇹, Republic of Ital...",test_unseen_entity,String
347978,infoseek_test_00347978,oven_05141568,Who occupies this place?,[Ministry of Communications and Multimedia],"[Ministry of Information, Communication and Cu...",test_unseen_entity,String


In [8]:
train_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_train_00000000,oven_01963180,Which place is this animal endemic to?,[People's Republic of China],"[cn, People's Republic of China, China, Mainla...",train,String,Q33602,Giant panda
1,infoseek_train_00000001,oven_03952028,What is the mohs' hardness of this material?,[7.6],"[{'wikidata': 7.6, 'range': [6.84, 8.36]}]",train,Numerical,Q43513,Emerald
2,infoseek_train_00000002,oven_01857671,What is the conservation status of this animal...,[Endangered],"[Endangered species, Endangered, EN]",train,String,Q192710,Malayan tapir
3,infoseek_train_00000003,oven_02959375,Who is the manufacturer of this vehicle?,[AM General],"[Am General, AM General LLC, AM General, AM Ge...",train,String,Q217046,Humvee
4,infoseek_train_00000004,oven_04416546,What fields are the person in the image specia...,[baking],[baking],train,String,Q160131,Baker
...,...,...,...,...,...,...,...,...,...
934043,infoseek_train_00934043,oven_00670496,How many days does the gestation of this anima...,[30.0-32.0],"[{'wikidata': 31.0, 'range': [30.0, 32.0]}]",train,Numerical,Q221612,Groundhog
934044,infoseek_train_00934044,oven_01683806,What is the oldest age of this animal?,[25],"[{'wikidata': 25.0, 'range': [22.5, 27.5000000...",train,Numerical,Q188657,Striped hyena
934045,infoseek_train_00934045,oven_00270092,What is the minimum number of players of a gam...,[2],"[{'wikidata': 2.0, 'range': [1.8, 2.2]}]",train,Numerical,Q7291,Badminton
934046,infoseek_train_00934046,oven_04651005,What is the source that produces this food?,[Quercus],"[oaks, oak, oak tree, the oak genus, Quercus]",train,String,Q3914781,Acorn


In [39]:
# convert list elements to tuple and do value counts
train_df[['image_id', 'question', 'answer']].apply(lambda x: tuple(x), axis=1).value_counts()

(oven_01963180, Which place is this animal endemic to?, [People's Republic of China])                                                                                                                                                                                                                            1
(oven_04140123, What is the closest upper taxonomy of this plant?, [Capsicum annuum])                                                                                                                                                                                                                            1
(oven_01804351, What is the main food source of this animal?, [herbaceous plant])                                                                                                                                                                                                                                1
(oven_00198130, What is the country of origin of this food?, [People's Republic

In [51]:
# convert answer to tuple
train_df['answer'] = train_df['answer'].apply(lambda x: tuple(x))
# according to 'image_id', 'question', 'answer' drop duplicates
train_df.drop_duplicates(subset=['image_id', 'question', 'answer'], keep='first')

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_train_00000000,oven_01963180,Which place is this animal endemic to?,"(People's Republic of China,)","[cn, People's Republic of China, China, Mainla...",train,String,Q33602,Giant panda
1,infoseek_train_00000001,oven_03952028,What is the mohs' hardness of this material?,"(7.6,)","[{'wikidata': 7.6, 'range': [6.84, 8.36]}]",train,Numerical,Q43513,Emerald
2,infoseek_train_00000002,oven_01857671,What is the conservation status of this animal...,"(Endangered,)","[Endangered species, Endangered, EN]",train,String,Q192710,Malayan tapir
3,infoseek_train_00000003,oven_02959375,Who is the manufacturer of this vehicle?,"(AM General,)","[Am General, AM General LLC, AM General, AM Ge...",train,String,Q217046,Humvee
4,infoseek_train_00000004,oven_04416546,What fields are the person in the image specia...,"(baking,)",[baking],train,String,Q160131,Baker
...,...,...,...,...,...,...,...,...,...
934043,infoseek_train_00934043,oven_00670496,How many days does the gestation of this anima...,"(30.0-32.0,)","[{'wikidata': 31.0, 'range': [30.0, 32.0]}]",train,Numerical,Q221612,Groundhog
934044,infoseek_train_00934044,oven_01683806,What is the oldest age of this animal?,"(25,)","[{'wikidata': 25.0, 'range': [22.5, 27.5000000...",train,Numerical,Q188657,Striped hyena
934045,infoseek_train_00934045,oven_00270092,What is the minimum number of players of a gam...,"(2,)","[{'wikidata': 2.0, 'range': [1.8, 2.2]}]",train,Numerical,Q7291,Badminton
934046,infoseek_train_00934046,oven_04651005,What is the source that produces this food?,"(Quercus,)","[oaks, oak, oak tree, the oak genus, Quercus]",train,String,Q3914781,Acorn


In [40]:
# convert list elements to tuple and do value counts
train_df[['image_id', 'question']].apply(lambda x: tuple(x), axis=1).value_counts()

(oven_00056339, Who designed this bridge?)                                                                                                                                                                                                                                                      2
(oven_00056334, Who designed this bridge?)                                                                                                                                                                                                                                                      2
(oven_00056336, Who designed this bridge?)                                                                                                                                                                                                                                                      2
(oven_00056325, Who designed this bridge?)                                                                                        

In [49]:
# check for each image_id if there are multiple entity_id
train_df[['image_id', 'entity_id']].drop_duplicates()['image_id'].value_counts()
train_df[train_df['image_id'] == 'oven_00000037']
# for each image_id + question there is only one entity_id + answer

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
163103,infoseek_train_00163103,oven_00000037,who/what operates this aircraft?,[Japan Coast Guard],"[JMSA, Japan Coast Guard, MSA, JCG, Maritime S...",train,String,Q1141409,Dassault Falcon 900
724545,infoseek_train_00724545,oven_00000037,What product does this organization produce?,[Dassault Mirage 2000],"[M2000, Dassault-Breguet Mirage 2000, Dassault...",train,String,Q460487,Dassault Aviation


In [58]:
# convert list elements to tuple and do value counts
pd.DataFrame(val_df[['image_id', 'question', 'entity_id', 'answer']].apply(lambda x: tuple(x), axis=1).value_counts()).to_csv("val_count.csv")

In [59]:
# convert answer to tuple
val_df['answer'] = val_df['answer'].apply(lambda x: tuple(x))
# according to 'image_id', 'question', 'answer' drop duplicates
val_df = val_df.drop_duplicates(subset=['image_id', 'question', 'answer'], keep='first')
val_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_val_00000000,oven_04990048,What is the objective of this object?,"(energy transformation,)","[energy conversion, energy transformation, con...",val_unseen_question,String,Q178185,Heat engine
1,infoseek_val_00000001,oven_04962816,What is the conservation status of this animal...,"(Endangered,)","[Endangered species, Endangered, EN]",val_unseen_question,String,Q173651,African wild dog
2,infoseek_val_00000002,oven_04967661,How many days does the gestation of this anima...,"(6.5-7.0,)","[{'wikidata': 6.75, 'range': [6.5, 7.0]}]",val_unseen_question,Numerical,Q132576,Impala
3,infoseek_val_00000003,oven_05001968,What is the source that produces this plant?,"(Vitis labrusca,)","[Vitis labrusca, Fox grape]",val_unseen_question,String,Q393047,Concord grape
4,infoseek_val_00000004,oven_04951981,What country does this place belong to?,"(Pakistan,)","[Islamic Republic of Pakistan, 🇵🇰, pk, PAK, Pa...",val_unseen_question,String,Q996962,Minar-e-Pakistan
...,...,...,...,...,...,...,...,...,...
73615,infoseek_val_00073615,oven_05023559,Which city or region does this building locate...,"(Kyffhäuserland, Bad Frankenhausen)","[Bad Frankenhausen/Kyffhäuser, Bad Frankenhaus...",val_unseen_entity,String,Q573639,Kyffhäuser Monument
73616,infoseek_val_00073616,oven_00004044,In which year did this aircraft come into serv...,"(1968,)","[1967, 1968, 1969]",val_unseen_entity,Time,Q683041,De Havilland Canada DHC-6 Twin Otter
73617,infoseek_val_00073617,oven_04949031,What type of cuisine is this food?,"(Italian cuisine,)","[cuisine of Italy, Italian cuisine, Italian co...",val_unseen_entity,String,Q20026,Spaghetti
73618,infoseek_val_00073618,oven_05026314,Which historic county does this facility belon...,"(County Dublin,)","[co Dublin, Dublin, co. Dublin, County Dublin,...",val_unseen_entity,String,Q917854,Mount Jerome Cemetery and Crematorium


In [61]:
val_df[['image_id', 'question']].value_counts()

image_id       question                                                          
oven_00000167  What is the length in metre is the aircraft in the image?             1
oven_05021590  What is the architectural style of this building?                     1
oven_05021593  What country does this building belong to?                            1
oven_05021592  Who occupies this building?                                           1
               What is the architectural style of this building?                     1
                                                                                    ..
oven_05009812  What country does this organization belong to?                        1
               What is the parent organization of this organization?                 1
               What is this organization named after?                                1
               What product does this organization produce?                          1
oven_05070447  What is the decomposition point o

In [13]:
val_df[(val_df['image_id'] == "oven_05026594") & (val_df['question'] == "Who designed this building?")]

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
70405,infoseek_val_00070405,oven_05026594,Who designed this building?,[Thomas Telford],[Thomas Telford],val_unseen_entity,String,Q581526,Menai Suspension Bridge
71993,infoseek_val_00071993,oven_05026594,Who designed this building?,[Thomas Telford],[Thomas Telford],val_unseen_entity,String,Q581526,Menai Suspension Bridge


In [63]:
# convert answer to tuple
test_df['answer'] = test_df['answer'].apply(lambda x: tuple(x))
# according to 'image_id', 'question', 'answer' drop duplicates
test_df = test_df.drop_duplicates(subset=['image_id', 'question', 'answer'], keep='first')
test_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type_x,question_type_y
0,infoseek_test_00000000,oven_05494604,What place inflows lake?,"(Aare, Lütschine)","[Lütschine, Lutschine, Aare, Aar]",test_unseen_question,String,String
1,infoseek_test_00000001,oven_05651031,What is the country of origin of this animal?,"(Scotland, United Kingdom)","[U K, Great Britain, Britain, The United Kingd...",test_unseen_question,String,String
2,infoseek_test_00000002,oven_05717674,What is the brand of this vehicle?,"(Ford,)","[Ford brand of vehicles, Ford]",test_unseen_question,String,String
3,infoseek_test_00000003,oven_05528107,Who is the owner of this building?,"(Cleveland,)","[Cleveland, Ohio, Cleveland, Cleveland, OH]",test_unseen_question,String,String
4,infoseek_test_00000004,oven_05504893,How many spans does this bridge have?,"(1,)","[{'wikidata': 1.0, 'range': [0.9, 1.1]}]",test_unseen_question,Numerical,Numerical
...,...,...,...,...,...,...,...,...
347975,infoseek_test_00347975,oven_05161211,Who commissioned this building?,"(Damian Hugo Philipp von Schönborn,)",[Damian Hugo Philipp von Schönborn],test_unseen_entity,String,String
347976,infoseek_test_00347976,oven_05279591,What is the closest parent taxonomy of this an...,"(Callospermophilus,)",[Callospermophilus],test_unseen_entity,String,String
347977,infoseek_test_00347977,oven_05454536,What country does this food belong to?,"(Italy,)","[Italy, Italian Republic, 🇮🇹, Republic of Ital...",test_unseen_entity,String,String
347978,infoseek_test_00347978,oven_05141568,Who occupies this place?,"(Ministry of Communications and Multimedia,)","[Ministry of Information, Communication and Cu...",test_unseen_entity,String,String


In [65]:
pd.DataFrame(test_df[['image_id', 'question']].value_counts()).to_csv("test_count.csv")

In [69]:
# drop samples that show more than once by 'image_id', 'question'
test_df = test_df.drop_duplicates(subset=['image_id', 'question'], keep=False)
test_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type_x,question_type_y
0,infoseek_test_00000000,oven_05494604,What place inflows lake?,"(Aare, Lütschine)","[Lütschine, Lutschine, Aare, Aar]",test_unseen_question,String,String
1,infoseek_test_00000001,oven_05651031,What is the country of origin of this animal?,"(Scotland, United Kingdom)","[U K, Great Britain, Britain, The United Kingd...",test_unseen_question,String,String
2,infoseek_test_00000002,oven_05717674,What is the brand of this vehicle?,"(Ford,)","[Ford brand of vehicles, Ford]",test_unseen_question,String,String
3,infoseek_test_00000003,oven_05528107,Who is the owner of this building?,"(Cleveland,)","[Cleveland, Ohio, Cleveland, Cleveland, OH]",test_unseen_question,String,String
4,infoseek_test_00000004,oven_05504893,How many spans does this bridge have?,"(1,)","[{'wikidata': 1.0, 'range': [0.9, 1.1]}]",test_unseen_question,Numerical,Numerical
...,...,...,...,...,...,...,...,...
347975,infoseek_test_00347975,oven_05161211,Who commissioned this building?,"(Damian Hugo Philipp von Schönborn,)",[Damian Hugo Philipp von Schönborn],test_unseen_entity,String,String
347976,infoseek_test_00347976,oven_05279591,What is the closest parent taxonomy of this an...,"(Callospermophilus,)",[Callospermophilus],test_unseen_entity,String,String
347977,infoseek_test_00347977,oven_05454536,What country does this food belong to?,"(Italy,)","[Italy, Italian Republic, 🇮🇹, Republic of Ital...",test_unseen_entity,String,String
347978,infoseek_test_00347978,oven_05141568,Who occupies this place?,"(Ministry of Communications and Multimedia,)","[Ministry of Information, Communication and Cu...",test_unseen_entity,String,String


In [62]:
# delete where question == Who designed this bridge? and entity_id == Q125821
train_df = train_df[~((train_df["question"] == "Who designed this bridge?") & (train_df["entity_id"] == "Q125821"))]
train_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_train_00000000,oven_01963180,Which place is this animal endemic to?,[People's Republic of China],"[cn, People's Republic of China, China, Mainla...",train,String,Q33602,Giant panda
1,infoseek_train_00000001,oven_03952028,What is the mohs' hardness of this material?,[7.6],"[{'wikidata': 7.6, 'range': [6.84, 8.36]}]",train,Numerical,Q43513,Emerald
2,infoseek_train_00000002,oven_01857671,What is the conservation status of this animal...,[Endangered],"[Endangered species, Endangered, EN]",train,String,Q192710,Malayan tapir
3,infoseek_train_00000003,oven_02959375,Who is the manufacturer of this vehicle?,[AM General],"[Am General, AM General LLC, AM General, AM Ge...",train,String,Q217046,Humvee
4,infoseek_train_00000004,oven_04416546,What fields are the person in the image specia...,[baking],[baking],train,String,Q160131,Baker
...,...,...,...,...,...,...,...,...,...
934043,infoseek_train_00934043,oven_00670496,How many days does the gestation of this anima...,[30.0-32.0],"[{'wikidata': 31.0, 'range': [30.0, 32.0]}]",train,Numerical,Q221612,Groundhog
934044,infoseek_train_00934044,oven_01683806,What is the oldest age of this animal?,[25],"[{'wikidata': 25.0, 'range': [22.5, 27.5000000...",train,Numerical,Q188657,Striped hyena
934045,infoseek_train_00934045,oven_00270092,What is the minimum number of players of a gam...,[2],"[{'wikidata': 2.0, 'range': [1.8, 2.2]}]",train,Numerical,Q7291,Badminton
934046,infoseek_train_00934046,oven_04651005,What is the source that produces this food?,[Quercus],"[oaks, oak, oak tree, the oak genus, Quercus]",train,String,Q3914781,Acorn


In [73]:
val_df[~((val_df["question"] == "Who designed this bridge?") & (val_df["entity_id"] == "Q125821"))]

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_val_00000000,oven_04990048,What is the objective of this object?,[energy transformation],"[energy conversion, energy transformation, con...",val_unseen_question,String,Q178185,Heat engine
1,infoseek_val_00000001,oven_04962816,What is the conservation status of this animal...,[Endangered],"[Endangered species, Endangered, EN]",val_unseen_question,String,Q173651,African wild dog
2,infoseek_val_00000002,oven_04967661,How many days does the gestation of this anima...,[6.5-7.0],"[{'wikidata': 6.75, 'range': [6.5, 7.0]}]",val_unseen_question,Numerical,Q132576,Impala
3,infoseek_val_00000003,oven_05001968,What is the source that produces this plant?,[Vitis labrusca],"[Vitis labrusca, Fox grape]",val_unseen_question,String,Q393047,Concord grape
4,infoseek_val_00000004,oven_04951981,What country does this place belong to?,[Pakistan],"[Islamic Republic of Pakistan, 🇵🇰, pk, PAK, Pa...",val_unseen_question,String,Q996962,Minar-e-Pakistan
...,...,...,...,...,...,...,...,...,...
73615,infoseek_val_00073615,oven_05023559,Which city or region does this building locate...,"[Kyffhäuserland, Bad Frankenhausen]","[Bad Frankenhausen/Kyffhäuser, Bad Frankenhaus...",val_unseen_entity,String,Q573639,Kyffhäuser Monument
73616,infoseek_val_00073616,oven_00004044,In which year did this aircraft come into serv...,[1968],"[1967, 1968, 1969]",val_unseen_entity,Time,Q683041,De Havilland Canada DHC-6 Twin Otter
73617,infoseek_val_00073617,oven_04949031,What type of cuisine is this food?,[Italian cuisine],"[cuisine of Italy, Italian cuisine, Italian co...",val_unseen_entity,String,Q20026,Spaghetti
73618,infoseek_val_00073618,oven_05026314,Which historic county does this facility belon...,[County Dublin],"[co Dublin, Dublin, co. Dublin, County Dublin,...",val_unseen_entity,String,Q917854,Mount Jerome Cemetery and Crematorium


In [77]:
test_df[(test_df["question"] == "Who designed this bridge?")].to_csv("test_bridge.csv")

In [3]:
# save train_df, val_df, test_df to /infoseek
train_df.to_csv("infoseek/train.csv", index=False)
val_df.to_csv("infoseek/val.csv", index=False)
test_df.to_csv("infoseek/test.csv", index=False)


KeyboardInterrupt



In [4]:
# read train/val/test.csv
train_df = pd.read_csv("infoseek/train.csv")
val_df = pd.read_csv("infoseek/val.csv")
test_df = pd.read_csv("infoseek/test.csv")

In [5]:
# convert answer from tuple to list
train_df['answer'] = train_df['answer'].apply(lambda x: list(x))
val_df['answer'] = val_df['answer'].apply(lambda x: list(x))
test_df['answer'] = test_df['answer'].apply(lambda x: list(x))

In [6]:
train_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_train_00000000,oven_01963180,Which place is this animal endemic to?,"[[, "", P, e, o, p, l, e, ', s, , R, e, p, u, ...","['cn', ""People's Republic of China"", 'China', ...",train,String,Q33602,Giant panda
1,infoseek_train_00000001,oven_03952028,What is the mohs' hardness of this material?,"[[, ', 7, ., 6, ', ]]","[{'wikidata': 7.6, 'range': [6.84, 8.36]}]",train,Numerical,Q43513,Emerald
2,infoseek_train_00000002,oven_01857671,What is the conservation status of this animal...,"[[, ', E, n, d, a, n, g, e, r, e, d, ', ]]","['Endangered species', 'Endangered', 'EN']",train,String,Q192710,Malayan tapir
3,infoseek_train_00000003,oven_02959375,Who is the manufacturer of this vehicle?,"[[, ', A, M, , G, e, n, e, r, a, l, ', ]]","['Am General', 'AM General LLC', 'AM General',...",train,String,Q217046,Humvee
4,infoseek_train_00000004,oven_04416546,What fields are the person in the image specia...,"[[, ', b, a, k, i, n, g, ', ]]",['baking'],train,String,Q160131,Baker
...,...,...,...,...,...,...,...,...,...
933967,infoseek_train_00934043,oven_00670496,How many days does the gestation of this anima...,"[[, ', 3, 0, ., 0, -, 3, 2, ., 0, ', ]]","[{'wikidata': 31.0, 'range': [30.0, 32.0]}]",train,Numerical,Q221612,Groundhog
933968,infoseek_train_00934044,oven_01683806,What is the oldest age of this animal?,"[[, ', 2, 5, ', ]]","[{'wikidata': 25.0, 'range': [22.5, 27.5000000...",train,Numerical,Q188657,Striped hyena
933969,infoseek_train_00934045,oven_00270092,What is the minimum number of players of a gam...,"[[, ', 2, ', ]]","[{'wikidata': 2.0, 'range': [1.8, 2.2]}]",train,Numerical,Q7291,Badminton
933970,infoseek_train_00934046,oven_04651005,What is the source that produces this food?,"[[, ', Q, u, e, r, c, u, s, ', ]]","['oaks', 'oak', 'oak tree', 'the oak genus', '...",train,String,Q3914781,Acorn


In [13]:
# Define the desired subsample size
total_sample_size = 73545 * 2

# Step 1: Group by 'entity_id' and 'question'
grouped = train_df.groupby(['entity_id', 'question'])

# Step 2: Determine sampling sizes based on group proportions, but ensure at least one is left
def calculate_sample_size(group, total_size):
    proportion = len(group) / len(train_df)
    group_sample_size = int(proportion * total_size)
    return min(group_sample_size, len(group) - 1)  # Sample size or one less than the group size

# Calculate each group's sample size
group_sample_sizes = grouped.apply(calculate_sample_size, total_size=total_sample_size)

# Sample from each group using the calculated sizes
def sample_group(group, size):
    if size > 0:
        return group.sample(n=size, random_state=1)
    else:
        return pd.DataFrame()

subsample = pd.concat([sample_group(group, size) for (name, group), size in zip(grouped, group_sample_sizes)])

# Step 3: Determine the remaining DataFrame by excluding the subsample
remaining_df = train_df.drop(subsample.index)

# Optional: Verification
# Check that every 'entity_id' and 'question' pair in subsample also exists in remaining_df
verification = subsample[['entity_id', 'question']].apply(lambda row: tuple(row), axis=1)
remaining_verification = remaining_df[['entity_id', 'question']].apply(lambda row: tuple(row), axis=1)
verified = verification.isin(remaining_verification)

if verified.all():
    print("All sampled pairs are still represented in the remaining DataFrame.")
else:
    print("Some sampled pairs are not represented in the remaining DataFrame.")

# Output the subsample and remaining_df sizes for verification
print("Subsample size:", len(subsample))
print("Remaining set size:", len(remaining_df))


All sampled pairs are still represented in the remaining DataFrame.
Subsample size: 143166
Remaining set size: 790806


In [14]:
remaining_df[['entity_id', 'question']].value_counts()

entity_id  question                                                                                                                                                                                                                                                                     
Q7075      What is the position of the head of this facility shown?                                                                                                                                                                                                                         3564
Q11573     What is this object commonly used to measure?                                                                                                                                                                                                                                    1595
Q35127     What is the software engine employed by this organization?                                                                        

In [15]:
train_df[['entity_id', 'question']].value_counts()

entity_id  question                                                                                                                                                                                                                                                                     
Q7075      What is the position of the head of this facility shown?                                                                                                                                                                                                                         4230
Q11573     What is this object commonly used to measure?                                                                                                                                                                                                                                    1892
Q35127     What is the software engine employed by this organization?                                                                        

In [16]:
remaining_df

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
0,infoseek_train_00000000,oven_01963180,Which place is this animal endemic to?,"[[, "", P, e, o, p, l, e, ', s, , R, e, p, u, ...","['cn', ""People's Republic of China"", 'China', ...",train,String,Q33602,Giant panda
1,infoseek_train_00000001,oven_03952028,What is the mohs' hardness of this material?,"[[, ', 7, ., 6, ', ]]","[{'wikidata': 7.6, 'range': [6.84, 8.36]}]",train,Numerical,Q43513,Emerald
2,infoseek_train_00000002,oven_01857671,What is the conservation status of this animal...,"[[, ', E, n, d, a, n, g, e, r, e, d, ', ]]","['Endangered species', 'Endangered', 'EN']",train,String,Q192710,Malayan tapir
5,infoseek_train_00000005,oven_03002067,Which culture is associated with this vehicle?,"[[, ', I, n, u, i, t, ', ]]","['Inuit', 'Inuk', 'Inuits']",train,String,Q179246,Kayak
6,infoseek_train_00000006,oven_03957581,Who is the discoverer or inventor of this mate...,"[[, ', J, e, a, n, -, B, a, p, t, i, s, t, e, ...",['Jean-Baptiste Guimet'],train,String,Q219660,Ultramarine
...,...,...,...,...,...,...,...,...,...
933966,infoseek_train_00934042,oven_00656757,How many days does the gestation of this anima...,"[[, ', 6, 0, ., 0, -, 6, 4, ., 0, ', ]]","[{'wikidata': 62.0, 'range': [60.0, 64.0]}]",train,Numerical,Q44299,Coyote
933967,infoseek_train_00934043,oven_00670496,How many days does the gestation of this anima...,"[[, ', 3, 0, ., 0, -, 3, 2, ., 0, ', ]]","[{'wikidata': 31.0, 'range': [30.0, 32.0]}]",train,Numerical,Q221612,Groundhog
933969,infoseek_train_00934045,oven_00270092,What is the minimum number of players of a gam...,"[[, ', 2, ', ]]","[{'wikidata': 2.0, 'range': [1.8, 2.2]}]",train,Numerical,Q7291,Badminton
933970,infoseek_train_00934046,oven_04651005,What is the source that produces this food?,"[[, ', Q, u, e, r, c, u, s, ', ]]","['oaks', 'oak', 'oak tree', 'the oak genus', '...",train,String,Q3914781,Acorn


In [17]:
subsample

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
129583,infoseek_train_00129596,oven_00103672,What is this building named after?,"[[, ', F, r, a, n, c, i, s, , o, f, , A, s, ...","['San Francisco de Asis', 'Francis of Assisi',...",train,String,Q1000321,Mission San Francisco de Asís
674970,infoseek_train_00675031,oven_00103704,What is this building named after?,"[[, ', F, r, a, n, c, i, s, , o, f, , A, s, ...","['San Francisco de Asis', 'Francis of Assisi',...",train,String,Q1000321,Mission San Francisco de Asís
130206,infoseek_train_00130219,oven_00103705,What is this building named after?,"[[, ', F, r, a, n, c, i, s, , o, f, , A, s, ...","['San Francisco de Asis', 'Francis of Assisi',...",train,String,Q1000321,Mission San Francisco de Asís
525126,infoseek_train_00525175,oven_00103684,What is this building named after?,"[[, ', F, r, a, n, c, i, s, , o, f, , A, s, ...","['San Francisco de Asis', 'Francis of Assisi',...",train,String,Q1000321,Mission San Francisco de Asís
616477,infoseek_train_00616533,oven_00103697,What is this building named after?,"[[, ', F, r, a, n, c, i, s, , o, f, , A, s, ...","['San Francisco de Asis', 'Francis of Assisi',...",train,String,Q1000321,Mission San Francisco de Asís
...,...,...,...,...,...,...,...,...,...
779680,infoseek_train_00779749,oven_00034691,What country does this building belong to?,"[[, ', P, o, r, t, u, g, a, l, ', ]]","['PRT', 'Portugal', '🇵🇹', 'República Portugues...",train,String,Q999002,Praça do Comércio
924807,infoseek_train_00924883,oven_00034565,What country does this building belong to?,"[[, ', P, o, r, t, u, g, a, l, ', ]]","['PRT', 'Portugal', '🇵🇹', 'República Portugues...",train,String,Q999002,Praça do Comércio
34223,infoseek_train_00034227,oven_00034395,What country does this building belong to?,"[[, ', P, o, r, t, u, g, a, l, ', ]]","['PRT', 'Portugal', '🇵🇹', 'República Portugues...",train,String,Q999002,Praça do Comércio
643607,infoseek_train_00643666,oven_00034555,What country does this building belong to?,"[[, ', P, o, r, t, u, g, a, l, ', ]]","['PRT', 'Portugal', '🇵🇹', 'República Portugues...",train,String,Q999002,Praça do Comércio


In [19]:
remaining_df.to_csv("infoseek/train_remaining.csv", index=True)
# shuffle subsample
subsample = subsample.sample(frac=1, random_state=1)
# equally split subsample into two files and save as val_seen.csv and test_seen.csv
subsample.iloc[:len(subsample)//2].to_csv("infoseek/val_seen.csv", index=True)
subsample.iloc[len(subsample)//2:].to_csv("infoseek/test_seen.csv", index=True)


In [22]:
# entity
train_entity = set(train_df['entity_id'])
val_entity = set(val_df['entity_id'])
len(train_entity), len(val_entity)

(5549, 1794)

In [23]:
train_df['entity_id'].value_counts()

entity_id
Q202878     9442
Q2736       5125
Q80811      5051
Q4504       4689
Q7075       4230
            ... 
Q161473        2
Q2187594       2
Q2978738       2
Q2033039       2
Q287007        2
Name: count, Length: 5549, dtype: int64

In [24]:
val_df['entity_id'].value_counts()

entity_id
Q257298     580
Q390370     538
Q217109     515
Q517545     475
Q204871     453
           ... 
Q3487324      2
Q614880       2
Q212047       2
Q6130468      2
Q1940276      2
Name: count, Length: 1794, dtype: int64

In [26]:
train_df['question_type'].value_counts() / len(train_df)

question_type
String       0.751718
Numerical    0.204468
Time         0.043814
Name: count, dtype: float64

In [23]:
val_seen_df = pd.read_csv("infoseek/val_seen.csv")
test_seen_df = pd.read_csv("infoseek/test_seen.csv")

In [25]:
print(train_df['question_type'].value_counts() / len(train_df))
print(remaining_df['question_type'].value_counts() / len(remaining_df))
print(val_seen_df['question_type'].value_counts() / len(val_seen_df))
print(test_seen_df['question_type'].value_counts() / len(test_seen_df))

question_type
String       0.751698
Numerical    0.204485
Time         0.043817
Name: count, dtype: float64
question_type
String       0.752188
Numerical    0.204075
Time         0.043736
Name: count, dtype: float64
question_type
String       0.750010
Numerical    0.205789
Time         0.044200
Name: count, dtype: float64
question_type
String       0.747971
Numerical    0.207703
Time         0.044326
Name: count, dtype: float64


In [41]:
subsample_new['question_type'].value_counts() / len(subsample_new)

question_type
String       0.752904
Numerical    0.203640
Time         0.043456
Name: count, dtype: float64

In [27]:
val_df['question_type'].value_counts() / len(val_df)

question_type
String       0.737829
Numerical    0.215920
Time         0.046251
Name: count, dtype: float64

In [28]:
val_df['data_split'].value_counts()

data_split
val_unseen_entity      54964
val_unseen_question    18656
Name: count, dtype: int64

In [26]:
val_df_unseen_entity = val_df[val_df['data_split'] == "val_unseen_entity"]
val_df_unseen_question = val_df[val_df['data_split'] == "val_unseen_question"]

In [30]:
len(val_df_unseen_entity), len(val_df_unseen_question)

(54964, 18656)

In [34]:
# unseen entity
print(len(set(train_df['entity_id'])), len(set(val_df_unseen_entity['entity_id'])))
len(set(train_df['entity_id']).intersection(set(val_df_unseen_entity['entity_id'])))

5549 1192


0

In [27]:
# unseen entity
print(len(set(remaining_df['entity_id'])), len(set(val_df_unseen_entity['entity_id'])))
len(set(remaining_df['entity_id']).intersection(set(val_df_unseen_entity['entity_id'])))

5549 1192


0

In [48]:
# modify
print(len(set(train_df['question'])), len(set(val_df_unseen_entity['question'])))
len(set(train_df['question']).intersection(set(val_df_unseen_entity['question'])))

881 486


376

In [35]:
# unseen question
print(len(set(train_df['entity_id'])), len(set(val_df_unseen_question['entity_id'])))
len(set(train_df['entity_id']).intersection(set(val_df_unseen_question['entity_id'])))

5549 602


602

In [28]:
# unseen question
print(len(set(remaining_df['entity_id'])), len(set(val_df_unseen_question['entity_id'])))
len(set(remaining_df['entity_id']).intersection(set(val_df_unseen_question['entity_id'])))

5549 602


602

In [35]:
# val seen
print(len(set(remaining_df[['entity_id', 'question']].value_counts().index)), len(set(val_seen_df[['entity_id', 'question']].value_counts().index)))
print(len(set(remaining_df[['entity_id', 'question']].value_counts().index).intersection(set(val_seen_df[['entity_id', 'question']].value_counts().index))))

# test seen
print(len(set(remaining_df[['entity_id', 'question']].value_counts().index)), len(set(test_seen_df[['entity_id', 'question']].value_counts().index)))
print(len(set(remaining_df[['entity_id', 'question']].value_counts().index).intersection(set(test_seen_df[['entity_id', 'question']].value_counts().index))))

7996 6019
6019
7996 6031
6031


In [36]:
# when subsample, make sure every entity_id has at least one sample
remaining_df[['entity_id', 'question']].value_counts()

entity_id  question                                                                                                                                                                                                                                                                     
Q7075      What is the position of the head of this facility shown?                                                                                                                                                                                                                         3564
Q11573     What is this object commonly used to measure?                                                                                                                                                                                                                                    1595
Q35127     What is the software engine employed by this organization?                                                                        

In [37]:
# Define the desired subsample size as 10% of the remaining_df
total_sample_size = int(len(remaining_df) * 0.10)

# Step 1: Group by 'entity_id' and 'question'
grouped = remaining_df.groupby(['entity_id', 'question'])

# Step 2: Determine sampling sizes based on group proportions
def calculate_sample_size(group, total_size):
    proportion = len(group) / len(remaining_df)
    group_sample_size = max(1, int(proportion * total_size))  # Ensure at least one sample
    return group_sample_size  # Direct calculation without enforcing min or max limits

group_sample_sizes = grouped.apply(calculate_sample_size, total_size=total_sample_size)

# Step 3: Sample from each group using the calculated sizes
def sample_group(group, size):
    # Check to not exceed the group size
    if size > len(group):
        return group
    return group.sample(n=size, random_state=1)

subsample_new = pd.concat([sample_group(group, size) for (name, group), size in zip(grouped, group_sample_sizes)])

# Print outputs
print("Subsample size:", len(subsample_new))

# Optional: Verify proportions
# This step checks if the proportions in the subsample closely match the original proportions
original_props = grouped.size() / len(remaining_df)
subsample_props = subsample_new.groupby(['entity_id', 'question']).size() / len(subsample_new)

# Compare proportions: This is to ensure the distribution is as expected
print("Original proportions in remaining_df:")
print(original_props)
print("Proportions in subsample:")
print(subsample_props)

Subsample size: 76974
Original proportions in remaining_df:
entity_id  question                                                                 
Q1000321   What is this building named after?                                           0.000042
           Which city or region does this building locate in?                           0.000018
Q1002123   What is the closest parent taxonomy of this bird?                            0.000004
Q1002843   How many kilogram does this vehicle weigh?                                   0.000046
           What is the immediately prior item that this vehicle follows in a series?    0.000039
                                                                                          ...   
Q996962    Who designed this place?                                                     0.000082
Q997457    What is the closest upper taxonomy of this bird?                             0.000092
Q998315    What is the closest upper taxonomy of this animal?                  

In [34]:
len(set(remaining_df[['entity_id', 'question']].value_counts().index))

7996

In [38]:
len(set(subsample_new[['entity_id', 'question']].value_counts().index))

7996

In [40]:
# save susample_new to train_remaining_10%.csv
subsample_new.to_csv("infoseek/train_remaining_10%.csv", index=True)

In [43]:
# subsample 10% of val_seen_df and val_df to val_seen_10%.csv and val_unseen_10%.csv
val_seen_df = val_seen_df.sample(frac=0.1, random_state=1)
val_df_unseen_entity = val_df_unseen_entity.sample(frac=0.1, random_state=1)
val_seen_df.to_csv("infoseek/val_seen_10%.csv", index=True)
val_df_unseen_entity.to_csv("infoseek/val_unseen_10%.csv", index=True)

In [87]:
val_df = pd.read_csv("infoseek/val.csv")
val_df = val_df.sample(frac=0.1, random_state=1)
val_df.to_csv("infoseek/val_unseen_10%.csv", index=True)

In [44]:
len(train_df[['entity_id', 'question']].value_counts())

7996

In [49]:
print(len(set(train_df['question'])), len(set(val_df_unseen_question['question'])))
len(set(train_df['question']).intersection(set(val_df_unseen_question['question'])))

881 360


304

In [54]:
val_unseen_question_seen_entity = list(set(val_df_unseen_question['entity_id']))
sub_df = train_df[train_df['entity_id'].isin(val_unseen_question_seen_entity)]

In [56]:
len(set(sub_df['entity_id']))

602

In [55]:
print(len(set(sub_df['question'])), len(set(val_df_unseen_question['question'])))
len(set(sub_df['question']).intersection(set(val_df_unseen_question['question'])))

356 360


201

In [64]:
sub_df[['entity_id', 'question']], val_df_unseen_question[['entity_id', 'question']]

(       entity_id                                           question
 47       Q858459  how many year do these object in the image typ...
 59          Q525  What is the surface gravity of the place in me...
 68       Q542500           What country does this animal belong to?
 82        Q34706             where do you usually find this animal?
 83        Q14381                     Where is the lake inflow from?
 ...          ...                                                ...
 934022    Q35694            What is the litter size of this animal?
 934030    Q47542         What kind of effect does this animal have?
 934040    Q82738  How many days does the gestation of this anima...
 934043   Q221612  How many days does the gestation of this anima...
 934047    Q25332  What is the conservation status of this bird? ...
 
 [100589 rows x 2 columns],
       entity_id                                           question
 0       Q178185              What is the objective of this object?
 1    

In [71]:
pd.merge(sub_df[['entity_id', 'question']].drop_duplicates(), val_df_unseen_question[['entity_id', 'question']].drop_duplicates(), on=['entity_id', 'question'], how="inner")

,entity_id,question
0,Q465811,Who designed this park?


In [73]:
train_df[(train_df['entity_id'] == "Q465811") & (train_df['question'] == "Who designed this park?")]

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
11647,infoseek_train_00011647,oven_00031864,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
40971,infoseek_train_00040971,oven_00031896,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
99062,infoseek_train_00099062,oven_00031874,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
107087,infoseek_train_00107087,oven_00031863,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
127451,infoseek_train_00127451,oven_00031893,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
143533,infoseek_train_00143533,oven_00031867,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
180307,infoseek_train_00180307,oven_00031871,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
191163,infoseek_train_00191163,oven_00031875,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
221556,infoseek_train_00221556,oven_00031881,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall
234138,infoseek_train_00234138,oven_00031862,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",train,String,Q465811,National Mall


In [70]:
val_df_unseen_question[(val_df_unseen_question['entity_id'] == "Q465811") & (val_df_unseen_question['question'] == "Who designed this park?")]

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
1810,infoseek_val_00001810,oven_04952652,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
3256,infoseek_val_00003256,oven_04952655,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
3834,infoseek_val_00003834,oven_04952665,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
4805,infoseek_val_00004805,oven_04952654,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
5113,infoseek_val_00005113,oven_04952663,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
6093,infoseek_val_00006093,oven_04952656,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
6430,infoseek_val_00006430,oven_04952660,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
6927,infoseek_val_00006927,oven_04952661,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
8853,infoseek_val_00008853,oven_04952658,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall
9048,infoseek_val_00009048,oven_04952650,Who designed this park?,[Pierre Charles L'Enfant],"[Pierre C. L'Enfant, Pierre L'Enfant, Pierre-C...",val_unseen_question,String,Q465811,National Mall


In [14]:
js2df(split2data["val_withkb"]).head()

,data_id,entity_id,entity_text
0,infoseek_val_00042030,Q934551,Derwentwater
1,infoseek_val_00030546,Q1121020,Compact Disc Digital Audio
2,infoseek_val_00033267,Q155869,Whinchat
3,infoseek_val_00033700,Q20026,Spaghetti
4,infoseek_val_00005702,Q152073,Monserrate


In [85]:
pd.DataFrame(train_df[['image_id', 'question', 'entity_id']].value_counts()).to_csv("count.csv")

Repeating Samples
Question: Who designed this bridge?
Entity ID: Q125821

In [80]:
train_df[(train_df['image_id'] == "oven_00056329") & (train_df['question'] == "Who designed this bridge?")]

,data_id,image_id,question,answer,answer_eval,data_split,question_type,entity_id,entity_text
9152,infoseek_train_00009152,oven_00056329,Who designed this bridge?,[Cass Gilbert],[Cass Gilbert],train,String,Q125821,George Washington Bridge
55914,infoseek_train_00055914,oven_00056329,Who designed this bridge?,[Othmar Ammann],"[Othmar H. Ammann, Othmar Ammann, Othmar Herma...",train,String,Q125821,George Washington Bridge


# OVEN

In [36]:
oven = {
    "val_entity": "oven_data/oven_entity_val.jsonl",
    "val_query": "oven_data/oven_query_val.jsonl",
    "train_entity": "oven_data/oven_entity_train.jsonl",
    "train_query": "oven_data/oven_query_train.jsonl",
}

In [38]:
train_df_entity = js2df(oven["train_entity"])

In [40]:
val_df_entity = js2df(oven["val_entity"])
train_df_query = js2df(oven["train_query"])
val_df_query = js2df(oven["val_query"])

In [86]:
train_df_entity

,data_id,image_id,question,entity_id,entity_text,data_split
0,oven_entity_train_00000000,oven_00000000,what is the model of this aircraft?,Q1141409,Dassault Falcon 900,entity_train
1,oven_entity_train_00000001,oven_00000001,what is the model of this aircraft?,Q1141409,Dassault Falcon 900,entity_train
2,oven_entity_train_00000002,oven_00000002,what is the model of this aircraft?,Q1141409,Dassault Falcon 900,entity_train
3,oven_entity_train_00000003,oven_00000003,what is the model of this aircraft?,Q1141409,Dassault Falcon 900,entity_train
4,oven_entity_train_00000004,oven_00000004,what is the model of this aircraft?,Q1141409,Dassault Falcon 900,entity_train
...,...,...,...,...,...,...
4926309,oven_entity_train_04926309,oven_04925734,what is the main content of this image?,Q287,Wood,entity_train
4926310,oven_entity_train_04926310,oven_04925735,which type of material is depicted in the image?,Q287,Wood,entity_train
4926311,oven_entity_train_04926311,oven_04925736,what is shown in the photo?,Q287,Wood,entity_train
4926312,oven_entity_train_04926312,oven_04925737,what kind of material is this?,Q287,Wood,entity_train


In [90]:
train_df_query

,data_id,image_id,question,entity_id,entity_text,data_split
0,oven_query_train_00000000,oven_05420926,who should we vote for?,Q1623348,Hogan,query_train
1,oven_query_train_00000001,oven_05420927,what character is pictured on the cake?,Q188760,Hulk,query_train
2,oven_query_train_00000002,oven_05420928,what character is pictured on the tablecloth?,Q188760,Hulk,query_train
3,oven_query_train_00000003,oven_05420929,what is on little boys shirt?,Q188760,Hulk,query_train
4,oven_query_train_00000004,oven_05420930,what is on the horses legs?,Q1761743,Red tape,query_train
...,...,...,...,...,...,...
32250,oven_query_train_00032250,oven_05445155,what animal is shown?,Q5113,Bird,query_train
32251,oven_query_train_00032251,oven_05436952,what animal is in this picture?,Q5113,Bird,query_train
32252,oven_query_train_00032252,oven_05445156,what is flying?,Q5113,Bird,query_train
32253,oven_query_train_00032253,oven_05445157,what type of animal is pictured?,Q5113,Bird,query_train


In [87]:
val_df_entity

,data_id,image_id,question,entity_id,entity_text,data_split
0,oven_entity_val_00000000,oven_04944518,what is this park called?,Q517545,Nationals Park,entity_val_seen
1,oven_entity_val_00000001,oven_04944519,what is the name of this park?,Q517545,Nationals Park,entity_val_seen
2,oven_entity_val_00000002,oven_04944520,what is the name of this place?,Q517545,Nationals Park,entity_val_seen
3,oven_entity_val_00000003,oven_04944521,where is this park?,Q517545,Nationals Park,entity_val_seen
4,oven_entity_val_00000004,oven_04944522,where is this place?,Q517545,Nationals Park,entity_val_seen
...,...,...,...,...,...,...
126194,oven_entity_val_00126194,oven_05070493,what material is presented in the image?,Q134658,Niacin,entity_val_unseen
126195,oven_entity_val_00126195,oven_05070494,which type of material is depicted in the image?,Q134658,Niacin,entity_val_unseen
126196,oven_entity_val_00126196,oven_05070495,what is the category of this material?,Q134658,Niacin,entity_val_unseen
126197,oven_entity_val_00126197,oven_05070496,what material is presented in the image?,Q134658,Niacin,entity_val_unseen


In [88]:
val_df_query

,data_id,image_id,question,entity_id,entity_text,data_split
0,oven_query_val_00000000,oven_04925739,what type of bird are these?,Q755737,Killdeer,query_val_seen
1,oven_query_val_00000001,oven_04925740,what is the person using his foot for?,Q268534,Flush toilet,query_val_seen
2,oven_query_val_00000002,oven_04925741,what is the green vegetable on the plate?,Q37153,Avocado,query_val_seen
3,oven_query_val_00000003,oven_04925742,what is cut up in the bowl?,Q37153,Avocado,query_val_seen
4,oven_query_val_00000004,oven_04925743,what vegetable is on top of the rack?,Q37153,Avocado,query_val_seen
...,...,...,...,...,...,...
3286,oven_query_val_00003286,oven_04937912,what is the man holding?,Q457689,Surfboard,query_val_unseen
3287,oven_query_val_00003287,oven_04937913,what is the person holding?,Q457689,Surfboard,query_val_unseen
3288,oven_query_val_00003288,oven_04937914,what is the man riding?,Q457689,Surfboard,query_val_unseen
3289,oven_query_val_00003289,oven_04937915,what is the person on?,Q457689,Surfboard,query_val_unseen


In [43]:
len(train_df_entity), len(train_df_query)

(4926314, 32255)

In [47]:
len(set(list(val_df_entity['entity_id']) + list(val_df_query['entity_id'])))

3946

In [96]:
# unique query
len(set(list(val_df_entity['question']) + list(val_df_query['question'])))

3124

In [89]:
val_entity_unseen = val_df_entity[val_df_entity['data_split'] == "entity_val_unseen"]
val_entity_seen = val_df_entity[val_df_entity['data_split'] == "entity_val_seen"]

val_query_unseen = val_df_query[val_df_query['data_split'] == "query_val_unseen"]
val_query_seen = val_df_query[val_df_query['data_split'] == "query_val_seen"]

In [91]:
len(val_entity_unseen), len(val_entity_seen), len(val_query_unseen), len(val_query_seen)

(61098, 65101, 2268, 1023)

In [95]:
# seen entities: val_entity_seen + val_query_seen
len(set(list(val_entity_seen['entity_id']) + list(val_query_seen['entity_id'])))

1942

In [99]:
# unseen entities
len(set(list(val_entity_unseen['entity_id']) + list(val_query_unseen['entity_id'])))

2004

In [100]:
# seen
len(val_entity_seen) + len(val_query_seen)

66124

In [101]:
# unseen
len(val_entity_unseen) + len(val_query_unseen)

63366

In [103]:
train_oven = pd.concat([train_df_entity, train_df_query], axis=0)

In [104]:
# seen entity
print(len(set(train_oven['entity_id'])), len(set(val_entity_seen['entity_id'])))
len(set(train_oven['entity_id']).intersection(set(val_entity_seen['entity_id'])))

9788 1721


1712

In [121]:
# test if seen_entity and seen_query are wrt questions as well
print(len(train_oven[['entity_id', 'question']].drop_duplicates()), len(val_entity_seen[['entity_id', 'question']].drop_duplicates()))
len(pd.merge(train_oven[['entity_id', 'question']].drop_duplicates(), val_entity_seen[['entity_id', 'question']].drop_duplicates(), on=['entity_id', 'question'], how="inner"))

72898 10972


9933

In [120]:
print(len(train_oven[['entity_id', 'question']].drop_duplicates()), len(val_query_seen[['entity_id', 'question']].drop_duplicates()))
len(pd.merge(train_oven[['entity_id', 'question']].drop_duplicates(), val_query_seen[['entity_id', 'question']].drop_duplicates(), on=['entity_id', 'question'], how="inner"))

72898 968


165

In [105]:
# unseen entity
print(len(set(train_oven['entity_id'])), len(set(val_entity_unseen['entity_id'])))
len(set(train_oven['entity_id']).intersection(set(val_entity_unseen['entity_id'])))

9788 1471


0

In [106]:
# seen query
print(len(set(train_oven['entity_id'])), len(set(val_query_seen['entity_id'])))
len(set(train_oven['entity_id']).intersection(set(val_query_seen['entity_id'])))

9788 265


264

In [107]:
# unseen query
print(len(set(train_oven['entity_id'])), len(set(val_query_unseen['entity_id'])))
len(set(train_oven['entity_id']).intersection(set(val_query_unseen['entity_id'])))

9788 546


0

## For OVEN, the seen and unseen are only wrt entities

In [108]:
len(set(val_entity_seen['entity_id']).intersection(set(val_query_seen['entity_id'])))

44

In [113]:
same = list(set(val_entity_seen['entity_id']).intersection(set(val_query_seen['entity_id'])))
same

['Q177',
 'Q18398431',
 'Q16472452',
 'Q3487748',
 'Q1548030',
 'Q7639866',
 'Q11946202',
 'Q272502',
 'Q32489',
 'Q12337',
 'Q177941',
 'Q11111030',
 'Q165044',
 'Q34679',
 'Q31087',
 'Q446',
 'Q34706',
 'Q13317',
 'Q18545',
 'Q388450',
 'Q43663',
 'Q166080',
 'Q12493',
 'Q38965',
 'Q13188',
 'Q275447',
 'Q915026',
 'Q13450283',
 'Q8355',
 'Q1072900',
 'Q163446',
 'Q134768',
 'Q215857',
 'Q178024',
 'Q190868',
 'Q1192284',
 'Q5840',
 'Q13187',
 'Q160525',
 'Q208253',
 'Q605384',
 'Q168658',
 'Q5113',
 'Q7743']

In [114]:
val_entity_seen[val_entity_seen['entity_id'] == "Q177"]

,data_id,image_id,question,entity_id,entity_text,data_split
14834,oven_entity_val_00014834,oven_04959278,what kind of food is this?,Q177,Pizza,entity_val_seen
14835,oven_entity_val_00014835,oven_04959279,which category of food is shown in the image?,Q177,Pizza,entity_val_seen
14836,oven_entity_val_00014836,oven_04959280,which type of food is depicted in the image?,Q177,Pizza,entity_val_seen
14837,oven_entity_val_00014837,oven_04959281,what kind of food is this?,Q177,Pizza,entity_val_seen
14838,oven_entity_val_00014838,oven_04959282,which category of food is shown in the image?,Q177,Pizza,entity_val_seen
14839,oven_entity_val_00014839,oven_04959283,which type of food is depicted in the image?,Q177,Pizza,entity_val_seen
14840,oven_entity_val_00014840,oven_04959284,what food is presented in the image?,Q177,Pizza,entity_val_seen
14841,oven_entity_val_00014841,oven_04959285,what is this food?,Q177,Pizza,entity_val_seen
14842,oven_entity_val_00014842,oven_04959286,which category of food is shown in the image?,Q177,Pizza,entity_val_seen
14843,oven_entity_val_00014843,oven_04959287,which type of food is depicted in the image?,Q177,Pizza,entity_val_seen


In [115]:
val_query_seen[val_query_seen['entity_id'] == "Q177"]

,data_id,image_id,question,entity_id,entity_text,data_split
95,oven_query_val_00000095,oven_04925834,what type of food does ledo's sell?,Q177,Pizza,query_val_seen
96,oven_query_val_00000096,oven_04925835,what is hot-n-ready?,Q177,Pizza,query_val_seen
97,oven_query_val_00000097,oven_04925836,what company could be sponsoring this game?,Q177,Pizza,query_val_seen
98,oven_query_val_00000098,oven_04925837,what restaurant is in this theme park?,Q177,Pizza,query_val_seen
99,oven_query_val_00000099,oven_04925838,what can i put here?,Q177,Pizza,query_val_seen


In [41]:
tmp1 = set(train_df_entity['entity_id'])
tmp2 = set(val_df_entity['entity_id'])
print(len(tmp1), len(tmp2))
print(len(tmp1.intersection(tmp2)))

7943 3192
1604


In [42]:
tmp1 = set(train_df_entity['question'])
tmp2 = set(val_df_entity['question'])
print(len(tmp1), len(tmp2))
print(len(tmp1.intersection(tmp2)))

595 422
358


In [ ]:
# train_image_id_set = set(train_image_id)
# found = False  # Flag to track if any match is found

# val_image_id_new = []
# val_image_id_ = []
# for idx, split in enumerate(val_data_split):
#     if split == "val_unseen_question":
#         val_image_id_new.append(val_image_id[idx])
#     else:
#         val_image_id_.append(val_image_id[idx])

# train_image_id_set = set(train_image_id)
# val_image_id_set = set(val_image_id_new)
# val_image_id_set_ = set(val_image_id_)
# print("Number of train_image_id:", len(train_image_id_set))
# print("Number of val_image_id:", len(val_image_id_set))
# print("Number of val_image_id_:", len(val_image_id_set_))

# # 计算差集，找出仅在 val_image_id_set 中的元素
# unseen_in_train = val_image_id_set.difference(train_image_id_set)

# # 打印差集结果
# # print("Elements in val_image_id not in train_image_id:", unseen_in_train)

# # 判断是否所有val_image_id都在train_image_id中
# if not unseen_in_train:
#     print("All val_image_id elements are present in train_image_id.")
# else:
#     print(f"Not all val_image_id elements are present in train_image_id.  Not in # {len(unseen_in_train)}")

def select_random_subset(input_file, output_file, percentage=0.1):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    # Calculate the number of lines to sample
    sample_size = int(len(lines) * percentage)

    # Randomly sample lines
    sampled_lines = random.sample(lines, sample_size)

    # Write the selected lines to a new jsonl file
    with open(output_file, 'w') as outfile:
        for line in sampled_lines:
            outfile.write(line)

# Usage
# for split in ["train", "val", "test", "human"]:
#     select_random_subset(split2data[split], subset_output_path[split], 0.1)
